<a href="https://colab.research.google.com/github/annvorosh/GB/blob/MLB_FinalProject/MLB_L09_request.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Строим запрос к нашему сервису

In [1]:
import requests

import urllib.request
import json

In [5]:
# Пример данных
data = (
    "Why is women’s soccer so rare? It’s quite hard to find enough women willing to wear the same outfit."
)

In [6]:
# формируем запрос
def send_json(x):
    text = x
    #print(text)
    body = {
        'text': text
        }
    myurl = 'http://e87e-34-138-8-49.ngrok-free.app' + '/predict'
    #myurl = 'http://127.0.0.1:5000' + '/predict'
    headers = {'content-type': 'application/json; charset=utf-8'}
    response = requests.post(myurl, json=body, headers=headers)
    return response.json()['predictions']

In [7]:
# обращение к серверу с запросом из одного набора (его построили руками выше - data)
response = send_json(data)
print('предсказание', response)

предсказание 0.613468570582474


Сделаем обработку массы запросов:

1. загрузим данные X_test, y_test

https://drive.google.com/file/d/1b48idpnkdOAv5PAkbt7k1aQcBNupQU7v/view?usp=share_link

https://drive.google.com/file/d/1dvYmMrtXP4LlK81q_zHzduro1oJiA7gf/view?usp=share_link

In [14]:
!wget 'https://drive.google.com/uc?export=download&id=1b48idpnkdOAv5PAkbt7k1aQcBNupQU7v' -O X_test.csv
!wget 'https://drive.google.com/uc?export=download&id=1dvYmMrtXP4LlK81q_zHzduro1oJiA7gf' -O y_test.csv


--2023-07-04 15:49:06--  https://drive.google.com/uc?export=download&id=1b48idpnkdOAv5PAkbt7k1aQcBNupQU7v
Resolving drive.google.com (drive.google.com)... 172.253.115.102, 172.253.115.138, 172.253.115.139, ...
Connecting to drive.google.com (drive.google.com)|172.253.115.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-04-b8-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/smrsclc5nfvj27g8lgo1ajf0sgkta15r/1688485725000/06314451025212489701/*/1b48idpnkdOAv5PAkbt7k1aQcBNupQU7v?e=download&uuid=1a74a613-fc93-4ca3-a8d1-33210a42bc74 [following]
--2023-07-04 15:49:08--  https://doc-04-b8-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/smrsclc5nfvj27g8lgo1ajf0sgkta15r/1688485725000/06314451025212489701/*/1b48idpnkdOAv5PAkbt7k1aQcBNupQU7v?e=download&uuid=1a74a613-fc93-4ca3-a8d1-33210a42bc74
Resolving doc-04-b8-docs.googleusercontent.com (doc-04-b8-docs.googleusercontent.com)... 172.253.115.132, 

In [15]:
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score
from urllib import request, parse


X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

X_test.fillna('', inplace=True)

In [16]:
X_test.head()

,text,humor
0,Neighborhood full of million-dollar homes is n...,False
1,Life with 'devourers' and 'baby/bullies' -- yo...,False
2,Brazil's miss bumbum stirs controversy with bo...,False
3,How to get a job without experience ? just run...,True
4,Divorced celebrities who remarried after sayin...,False


In [17]:
X_test[['text']]

,text
0,Neighborhood full of million-dollar homes is n...
1,Life with 'devourers' and 'baby/bullies' -- yo...
2,Brazil's miss bumbum stirs controversy with bo...
3,How to get a job without experience ? just run...
4,Divorced celebrities who remarried after sayin...
...,...
65995,Pair of bulldogs prove teamwork makes the drea...
65996,"Whats black, white and red all over? 2 nuns in..."
65997,Ryan lochte touches anna wintour's knee at ral...
65998,The one question you should never have to ask ...


In [19]:
# response = send_json(X_test[['text']].iloc[0,:])
# response

import json

# Преобразовать Series в список или словарь
data = {"text": X_test['text'].iloc[0]}

# Преобразовать в JSON-строку
json_data = json.dumps(data)

# Отправить JSON-строку
response = send_json(json_data)
response

0.603186723633096

3. Сделаем N запросов и оценим время

In [29]:
N = 100

import time

%time
# predictions = X_test[['text']].iloc[:N].apply(lambda x: send_json(x), axis=1)


predictions = []

# Итерироваться по строкам
for _, row in X_test[['text']].iloc[:N].iterrows():
    # Получить значение столбца 'text'
    text = row['text']

    # Преобразовать значение в словарь или список
    data = {"text": text}

    # Преобразовать в JSON-строку
    json_data = json.dumps(data)

    # Отправить JSON-строку и получить результат
    result = send_json(json_data)

    # Добавить результат в список predictions
    predictions.append(result)


CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.72 µs


In [31]:
predictions = pd.Series(predictions)
predictions[:5].values

array([0.60318672, 0.28232544, 0.0704398 , 0.58001847, 0.36738792])

4. Посчитаем метрику

In [32]:
y_test.tail()

,humor
65995,False
65996,True
65997,False
65998,False
65999,False


In [33]:
import numpy as np

precision, recall, thresholds = precision_recall_curve(y_test[:N], predictions)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.756805695746634, F-Score=0.899, Precision=1.000, Recall=0.816


In [34]:
roc_auc_score(y_score=predictions.values, y_true=y_test.values[:N])

0.9611844737895158